# Predictive Modeling

Nicholas Greger

In [126]:
import math as mth
import numpy as np
from numpy import random

import pandas as pd
import pymc3 as pm

import scipy as sc
from scipy.stats import uniform, bernoulli, beta, expon, gamma

from bs4 import BeautifulSoup as soup
import requests
import urllib.request

import time

import pybaseball as pb

## Getting the Data

In order to be able to run the model, we'll need 3 statistics:
1. Starting pitcher Earned Run Average (ERA), deleniated for home and away.
2. Winning percent ratio
3. Team batting average

### Earned Running Average

In [138]:
pitching_2014h = pd.read_csv('2014_ERA_Home.csv')
pitching_2014a = pd.read_csv('2014_ERA_Away.csv')

pitching_2015h = pd.read_csv('2015_ERA_Home.csv')
pitching_2015a = pd.read_csv('2015_ERA_Away.csv')

pitching_2016h = pd.read_csv('2016_ERA_Home.csv')
pitching_2016a = pd.read_csv('2016_ERA_Away.csv')

pitching_2017h= pd.read_csv('2017_ERA_Home.csv')
pitching_2017a = pd.read_csv('2017_ERA_Away.csv')

In [173]:
ERA_2014 = pd.DataFrame()
ERA_2015 = pd.DataFrame()
ERA_2016 = pd.DataFrame()
ERA_2017 = pd.DataFrame()

ERA_2014['Team'] = pitching_2014h['Team']
ERA_2014['ERA_Home'] = pitching_2014h['ERA']
ERA_2014['ERA_Away'] = pitching_2014a['ERA']

ERA_2015['Team'] = pitching_2015h['Team']
ERA_2015['ERA_Home'] = pitching_2015h['ERA']
ERA_2015['ERA_Away'] = pitching_2015a['ERA']

ERA_2016['Team'] = pitching_2016h['Team']
ERA_2016['ERA_Home'] = pitching_2016h['ERA']
ERA_2016['ERA_Away'] = pitching_2016a['ERA']

ERA_2017['Team'] = pitching_2017h['Team']
ERA_2017['ERA_Home'] = pitching_2017h['ERA']
ERA_2017['ERA_Away'] = pitching_2017a['ERA']

In [178]:
# Replacing Team name with their abbreviations
era_list = [ERA_2014, ERA_2015, ERA_2016, ERA_2017]
team_names = ['Angels', 
         'Astros', 
         'Athletics', 
         'Blue Jays',
         'Braves',
         'Brewers',
         'Cardinals',
         'Cubs',
         'Diamondbacks',
         'Dodgers',
         'Giants',
         'Indians',
         'Mariners',
         'Marlins',
         'Mets',
         'Nationals',
         'Orioles',
         'Padres',
         'Phillies',
         'Pirates',
         'Rangers',
         'Rays',
         'Red Sox',
         'Reds',
         'Rockies',
         'Royals',
         'Tigers',
         'Twins',
         'White Sox',
         'Yankees']
team_abvr = ['LAA',
              'HOU',
              'OAK',
              'TOR',
              'ATL',
              'MIL',
              'STL',
              'CHC',
              'ARI',
              'LAD',
              'SFG',
              'CLE',
              'SEA',
              'MIA',
              'NYM',
              'WSN',
              'BAL',
              'SDP',
              'PHI',
              'PIT',
              'TEX',
              'TBR',
              'BOS',
              'CIN',
              'COL',
              'KCR',
              'DET',
              'MIN',
              'CHW',
              'NYY']

for dataframe in era_list:
    dataframe.replace(team_names, team_abvr, inplace=True)

In [179]:
for df in era_list:
    df = df.sort_values('Team')
    df.reset_index()

### Win Percent Ratio

In [101]:
standings_2014 = pd.read_csv('2014_standings.csv')
standings_2015 = pd.read_csv('2015_standings.csv')
standings_2016 = pd.read_csv('2016_standings.csv')
standings_2017 = pd.read_csv('2017_standings.csv')

In [103]:
wl_2014 = standings_2014[['Tm', 'W-L%']]
wl_2015 = standings_2015[['Tm', 'W-L%']]
wl_2016 = standings_2016[['Tm', 'W-L%']]
wl_2017 = standings_2017[['Tm', 'W-L%']]

### Batting Average

In [40]:
batting_2014 = pd.read_csv('2014_batting.csv')
batting_2015 = pd.read_csv('2015_batting.csv')
batting_2016 = pd.read_csv('2016_batting.csv')
batting_2017 = pd.read_csv('2017_batting.csv')

In [41]:
batting_average_2014 = batting_2014[['Tm', 'BA']]
batting_average_2015 = batting_2015[['Tm', 'BA']]
batting_average_2016 = batting_2016[['Tm', 'BA']]
batting_average_2017 = batting_2017[['Tm', 'BA']]

### Combined

In [180]:
data_2014 = pd.DataFrame()
data_2015 = pd.DataFrame()
data_2016 = pd.DataFrame()
data_2017 = pd.DataFrame()

data_2014[['Team', 'ERA_Home', 'ERA_Away']] = ERA_2014[['Team', 'ERA_Home', 'ERA_Away']]
data_2014['W/L%'] = wl_2014['W-L%']
data_2014['BA'] = batting_average_2014['BA']

data_2015[['Team', 'ERA_Home', 'ERA_Away']] = ERA_2015[['Team', 'ERA_Home', 'ERA_Away']]
data_2015['W/L%'] = wl_2015['W-L%']
data_2015['BA'] = batting_average_2015['BA']

data_2016[['Team', 'ERA_Home', 'ERA_Away']] = ERA_2016[['Team', 'ERA_Home', 'ERA_Away']]
data_2016['W/L%'] = wl_2016['W-L%']
data_2016['BA'] = batting_average_2016['BA']

data_2017[['Team', 'ERA_Home', 'ERA_Away']] = ERA_2017[['Team', 'ERA_Home', 'ERA_Away']]
data_2017['W/L%'] = wl_2017['W-L%']
data_2017['BA'] = batting_average_2017['BA']

In [186]:
data_2016

,Team,ERA_Home,ERA_Away,W/L%,BA
8,ARI,5.54,4.61,0.537,0.275
4,ATL,4.39,4.63,0.574,0.256
16,BAL,3.81,4.64,0.491,0.251
22,BOS,4.30,3.69,0.438,0.235
7,CHC,2.72,3.60,0.549,0.262
28,CHW,3.75,4.50,0.420,0.248
23,CIN,4.63,5.21,0.426,0.259
11,CLE,3.90,3.81,0.531,0.261
24,COL,5.40,4.40,0.426,0.258
26,DET,4.18,4.30,0.420,0.243


### Games

In [49]:
from pybaseball import schedule_and_record
HOU_2014 = schedule_and_record(2014, "HOU")
HOU_2015 = schedule_and_record(2015, "HOU")
HOU_2016 = schedule_and_record(2016, "HOU")
HOU_2017 = schedule_and_record(2017, "HOU")

In [116]:
from pybaseball import pitching_stats
data = pitching_stats(2014, 2017, ind=1)


,Season,Name,Team,Age,W,L,ERA,WAR,G,GS,...,wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi)
262,2015.0,Clayton Kershaw,Dodgers,27.0,16.0,7.0,2.13,8.6,33.0,33.0,...,1.76,22.85,0.365,0.665,0.511,0.478,0.811,0.689,0.487,23.4
191,2014.0,Clayton Kershaw,Dodgers,26.0,21.0,3.0,1.77,7.9,27.0,27.0,...,2.62,NaN,0.371,0.670,0.525,0.536,0.831,0.730,0.515,23.7
572,2017.0,Chris Sale,Red Sox,28.0,17.0,8.0,2.90,7.6,32.0,32.0,...,0.67,NaN,0.373,0.619,0.499,0.540,0.798,0.704,0.513,20.9
296,2017.0,Corey Kluber,Indians,31.0,18.0,4.0,2.25,7.2,29.0,29.0,...,4.65,9.31,0.388,0.595,0.489,0.433,0.853,0.681,0.485,23.5
353,2014.0,Corey Kluber,Indians,28.0,18.0,9.0,2.44,7.2,34.0,34.0,...,3.92,NaN,0.339,0.598,0.470,0.485,0.886,0.744,0.507,24.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2530,2017.0,Amir Garrett,Reds,25.0,3.0,8.0,7.39,-1.1,16.0,14.0,...,-0.88,NaN,0.281,0.643,0.452,0.645,0.888,0.808,0.472,19.2
2556,2017.0,Dylan Covey,White Sox,25.0,0.0,7.0,7.71,-1.1,18.0,12.0,...,-1.92,NaN,0.251,0.652,0.428,0.745,0.915,0.860,0.441,21.8
2317,2016.0,Chris Young,Royals,37.0,3.0,9.0,6.19,-1.2,34.0,13.0,...,0.33,NaN,0.320,0.621,0.450,0.597,0.855,0.751,0.432,20.9
2361,2017.0,Chris Beck,White Sox,26.0,2.0,1.0,6.40,-1.2,57.0,0.0,...,-0.92,NaN,0.236,0.684,0.454,0.696,0.892,0.840,0.488,29.8


In [124]:
data.groupby('Name')

## Initializing hyperparameters

In [217]:
def Initialize_Variables(era_home, 
                         era_away,
                         winning_ratio,
                         batting_average,
                         a_i = 2,
                         delta_zero = 0,
                         delta_one = 2,
                         m_0 = 10):
    """Initializes variables in the model in order to calculate the values using Gibbs
    
    *** Data ***
    ERA_*:              List |
    winning_ratio:      List | 
    batting_average:    List |
    a_i:                int  | Used to calculate r_i such that r_i ~ uniform(0, a_i)
    
    Returns:
        lambda_s: necessary in order to calculate the variables used in the Gibbs sampler. 
    """
    
    ## Step 1 ##
    
    print("Calculating hyperparameters... ")
    
    # Number of teams
    number_of_teams = len(batting_average)
    
    # Covariates
    alpha_s = [] # alpha_s
    for teams in winning_ratio:
        alpha_s.append(teams)
        
    beta_s = []
    for team in batting_average:
        beta_s.append(team)
    
    gamma_s = []
    for player in era_home:
        gamma_s.append(player)
    
    D_s = []
    for t in range(number_of_teams):
        D_s.append([alpha_s[t], beta_s[t], gamma_s[t]])
    
    r_i = uniform(0, a_i)
    r = np.random.uniform(0, a_i)
    
    lambda_s = []
    for i in range(number_of_teams):
        lambda_s.append(alpha_s[i]**r * beta_s[i]**r * gamma_s[i]**r)
    
    delta = np.random.uniform(delta_zero, delta_one)
    
    p_s = []
    for lambda_t in lambda_s:
        p_s.append(sc.special.beta(lambda_t*delta, 1))
    
    m = sc.stats.expon.pdf(m_0)
    
    return lambda_s, m, delta

In [218]:
Initialize_Variables(era_home = data_2016['ERA_Home'],
                     era_away = data_2016['ERA_Away'],
                     winning_ratio = data_2016['W/L%'],
                     batting_average = data_2016['BA'])

Calculating hyperparameters... 


([0.703740767924755,
  0.46429331371003185,
  0.2663040973688715,
  0.24013246651095985,
  0.19350709011831552,
  0.19295397089900485,
  0.30864028012991374,
  0.3406814688380922,
  0.40127974941275035,
  0.22515693148495652,
  0.3057020010429492,
  0.25640507371797355,
  0.4872875100474122,
  0.22444138218342424,
  0.2523286958482352,
  0.3435182283531075,
  0.36634353203644765,
  0.26137042068559024,
  0.17544805519801965,
  0.4307954692856378,
  0.2755426781285108,
  0.2705545022911179,
  0.2936270394402094,
  0.32485112888457934,
  0.26762334641079677,
  0.3778164493570939,
  0.23809234770035406,
  0.28272869436884623,
  0.5029186175039737,
  0.21768194567047147],
 4.5399929762484854e-05,
 1.677097523122112)

## Gibbs Sampling

In order to generate accurate priors, we need to use a Gibbs sampler to sample from the conditional densitiy. The priors are then determined as the point of which the algorithm converges. 

### Posterior Density

$$p[\cdot | \cdot] \propto \left(\prod_{s=t_0}^{t-1} B(m\lambda_s\delta, m)p_s^{x_s+m\lambda_s\delta-1}(1-p_s)^{m-x_s}\right)e^{\frac{-m}{m_0}}$$

### $$P_s$$

$$ p_s \sim beta(m\lambda_s\delta, m)$$

Calculating $p_s$ is simpler than the rest of the densities as $p_s$ is simply calculated using the beta function. 

In [228]:
def Calculate_P(m, lambda_s, delta):
    """
    Calculates p_s
    returns:
        p_s
    """
    p_s = []
    for lambda_t in lambda_s:
        p_s.append(sc.special.beta(m*lambda_t*delta, m))
    return p_s

### $$ m $$


$$P[m | \cdot	] \propto e^{\frac{-m}{m_0}}\prod_{s = t_0}^{t-1}B(m\lambda_s\delta, m)p_s^{m\lambda_s\delta}(1 - p_s)^m$$ where $B(a, b) = \frac{\Gamma(a+b)}{\Gamma(a)\Gamma(b)}$

In [2]:
# Used in the Gibbs Sampler for M
def B(a, b):
    return sc.special.gamma(a + b)/(sc.special.gamma(a)*sc.special.gamma(b))

In [1]:
def Calculate_M(lambda_s, p_s, delta, m_0, N = 10000):
    """
    Conditional Density for m
    returns: 
        m
    """
    m = np.expon(m_0)
    m_samples = []
    for samples in range(N):
        for lambda_t, p_t in zip(lambda_s, p_s):
            m_sampled = B(m*lambda_t*delta, m)*p_t**(m*lambda_t*delta)*(1-p_t)**m
            m_samples.append(m_sampled)
            m = np.prod(m_samples)
            
    return e**((-1 * m)/m_0)*m_samples

### $$\delta, r_i$$

$$P[\delta | \cdot	] \propto \prod_{s=t_0}^{t-1} \frac{\Gamma(m\lambda_s\delta+m)}{\Gamma(m\lambda_s\delta)}p_s^{m\lambda_s\delta} $$

In [35]:
def Calculate_Delta(lambda_s,
                    p_s,
                    m,
                    delta_zero,
                    delta_one,
                    N = 10000):
    """
    Calculates delta using Metropolis-within-Gibbs
    returns:
        delta
    """
    
    delta_samples = [] # All samples received
    
    delta = np.random.uniform(delta_zero, delta_one)
    for n_iter in range(N):
        for lambda_t, p_t in zip(lambda_s, p_s):
            delta_sampled = sc.special.gamma(m*lambda_t*delta+m)/sc.special.gamma(m*lambda_t*delta)*p_t**(m*lambda_t*delta)
            delta_samples.append(delta_sampled)
            delta = np.prod(delta_samples)
    return delta_samples

$$P[r | \cdot	 ] \propto \prod_{s=t_0}^{t-1} \frac{\Gamma(m\lambda_s\delta+m)}{\Gamma(m\lambda_s\delta)}p_s^{m\lambda_s\delta} $$

In [ ]:
def Calculate_R(lambda_s,
                    p_s,
                    m,
                    a_i,
                    delta, 
                    N = 10000):
    """
    Calculates r_i using Metropolis-within-Gibbs
    returns:
        r
    """
    r_totals
    r_samples = [] # All samples received
    
    r = np.random.uniform(0, a_i)
    for n_iter in range(N):
        for lambda_t, p_t in zip(lambda_s, p_s):
            r_sampled = sc.special.gamma(m*lambda_t*delta+m)/sc.special.gamma(m*lambda_t*delta)*p_t**(m*lambda_t*delta)
            r_samples.append(r_sampled)
            r = np.prod(r_samples)
    return r_samples

### Calculating the variables

In [229]:
initialized_variables = Initialize_Variables(era_home = data_2016['ERA_Home'],
                     era_away = data_2016['ERA_Away'],
                     winning_ratio = data_2016['W/L%'],
                     batting_average = data_2016['BA'])

Calculate_P(lambda_s = initialized_variables[0], m = initialized_variables[1], delta = initialized_variables[2])

Calculating hyperparameters... 


[40328.56351397958,
 49013.693561419685,
 67377.10764604436,
 71976.3853628005,
 83131.81909673152,
 83295.36587596763,
 61540.60212041993,
 58059.238097379995,
 52951.341995992974,
 75071.91520525631,
 61895.128344835175,
 69009.93382535894,
 47822.67120447438,
 75229.81365046623,
 69718.30466694316,
 57781.312589705885,
 55696.73778046151,
 68175.96055828061,
 88985.5513942001,
 50968.27122952326,
 65955.67427798081,
 66711.48968884799,
 63424.020687171884,
 59696.25390300883,
 67168.32416226626,
 54741.041664685916,
 72375.92588529212,
 64912.20584262637,
 47073.228300507086,
 76770.88935400538]

## Predicting Games

In [25]:
def PredictWinners(m, lambda_s, delta, m, p_s, games):
    for i in range(games):
        